## 1.- Descargar e instalar Spark con Hadoop



1- 
Instalación java jdk 8
Descarga de Spark 2.4.8 con hadoop 2.7
Instalar las librerías pyspark, findspark

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
!tar xf spark-2.4.8-bin-hadoop2.7.tgz
!pip install -q pyspark
!pip install -q findspark

tar: spark-2.4.8-bin-hadoop2.7.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now
     |████████████████████████████████| 281.3 MB 48 kB/s 
     |████████████████████████████████| 199 kB 32.9 MB/s 


2.- Crear variables de entorno

In [2]:
import os # libreria de manejo del sistema operativo
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"
os.environ["PYTHON_PATH"] = "/content/spark-2.4.8-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip"

3.- Creación de sesión python SparkSession y SparkContext

In [ ]:
# findspark permite usar pyspark (interfaz de Python a Spark),
# desde cualquier programa escrito en Python.
import findspark
findspark.init()
import pandas as pd
import pyspark
from pyspark.sql import SparkSession
from pyspark  import SparkContext
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc=SparkContext.getOrCreate()
sc

<SparkContext master=local[*] appName=pyspark-shell>

##2.- Carga y Análisis de Dataframe

###1 Forma.- Crear Dataframe con Esquema de Datos
El esquema de datos lo infiere SparkSession 



In [ ]:
archivo = './sample_data/Titanic.csv'
df= (spark.read
     .format("csv")
     .option('header','true')
     .option('inferSchema','true')
     .option('sep',',')
     .load(archivo))

Ver esquema de datos dataframe

In [ ]:
df.dtypes

[('PassengerId', 'int'),
 ('Survived', 'int'),
 ('Pclass', 'int'),
 ('Name', 'string'),
 ('Sex', 'string'),
 ('Age', 'double'),
 ('SibSp', 'int'),
 ('Parch', 'int'),
 ('Ticket', 'string'),
 ('Fare', 'double'),
 ('Cabin', 'string'),
 ('Embarked', 'string')]

###2 Forma: Crear Dataframe por esquema de datos por defecto

Creación dataframe por defecto

In [ ]:
archivo = './sample_data/Titanic.csv'
df= (spark.read
     .format("csv")
     .option('header','true')
     .load(archivo))

Ver el tipo de datos del Dataframe

In [ ]:
df.dtypes

[('PassengerId', 'string'),
 ('Survived', 'string'),
 ('Pclass', 'string'),
 ('Name', 'string'),
 ('Sex', 'string'),
 ('Age', 'string'),
 ('SibSp', 'string'),
 ('Parch', 'string'),
 ('Ticket', 'string'),
 ('Fare', 'string'),
 ('Cabin', 'string'),
 ('Embarked', 'string')]

Cambiar el esquema de datos 

In [ ]:
from pyspark.sql.functions import col

df = df.select(col('PassengerId').cast('int'),
                         col('Survived').cast('int'),
                         col('Pclass').cast('float'),
                         col('Name'),
                         col('Sex'),
                         col('Age').cast('float'),
                         col('SibSp').cast('float'),
                         col('Parch').cast('float'),
                         col('Ticket'),
                         col('Fare').cast('float'),
                         col('Cabin'),
                        col('Embarked')
                        )
df.dtypes



[('PassengerId', 'int'),
 ('Survived', 'int'),
 ('Pclass', 'float'),
 ('Name', 'string'),
 ('Sex', 'string'),
 ('Age', 'float'),
 ('SibSp', 'float'),
 ('Parch', 'float'),
 ('Ticket', 'string'),
 ('Fare', 'float'),
 ('Cabin', 'string'),
 ('Embarked', 'string')]

## 3.- Exploración de los datos 

Ver el conjunto de datos de dataframe

In [ ]:
df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

Count
Muestra la cantidad de filas de Dataframe

In [ ]:
df.count()

891

columns
Muestra las columnas del dataframe

In [ ]:
df.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

Describe
Entrega count,means,min,max de valores númericos de dataframe

In [ ]:
df.describe().show()

+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|                Name|   Sex|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               714|               891|                891|               891|              891|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                null|  null| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.20420

First
Entrega la primera fila del dataframe

In [ ]:
df.first()

Row(PassengerId=1, Survived=0, Pclass=3, Name='Braund, Mr. Owen Harris', Sex='male', Age=22.0, SibSp=1, Parch=0, Ticket='A/5 21171', Fare=7.25, Cabin=None, Embarked='S')

Distinct
Muestra los distintos valores de una columna

In [ ]:
df.select("Age").distinct().show()

+----+
| Age|
+----+
| 8.0|
|70.0|
| 7.0|
|20.5|
|49.0|
|29.0|
|40.5|
|64.0|
|47.0|
|42.0|
|24.5|
|44.0|
|35.0|
|null|
|62.0|
|18.0|
|80.0|
|34.5|
|39.0|
| 1.0|
+----+
only showing top 20 rows



##4.- Preprocesado de los datos

####1.-  Ver Balanceo de las clase a clasificar

In [ ]:
from pyspark.sql.functions import col, count
Clase=(df.groupBy("Survived")
  .agg(
      count("Survived").alias("count")
      )
  )
Clase.show()

+--------+-----+
|Survived|count|
+--------+-----+
|       1|  342|
|       0|  549|
+--------+-----+



### 2.- Identificar Etiqueta en el modelo de datos 

In [ ]:
df = df.withColumnRenamed("Survived", "label")

### 3.- Determinar los valores null y nan en las columnas

In [ ]:
from pyspark.sql.functions import isnull,isnan,when,count,col
df.select([count(when(isnan(c)|isnull(c), c)).alias(c) for c in df.columns]).show()

+-----------+-----+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|label|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+-----+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|    0|     0|   0|  0|177|    0|    0|     0|   0|  687|       2|
+-----------+-----+------+----+---+---+-----+-----+------+----+-----+--------+



###4.-. Convertir los valores 0 en nan en columnas que tenga sentido hacerlo

In [ ]:
import numpy as np
from pyspark.sql.functions import when

df = df.withColumn("Parch",when(df.Parch==0,np.nan).otherwise(df.Parch))

###5.- Reemplazar valores nullos de Dataset

fillna()
Función que permite reemplazar los valores nulos de una columna por un valor determinado por el usuario

In [ ]:
df = df.fillna("N/A", ["Cabin"])
df.show()

+-----------+-----+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|label|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+-----+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|    0|     3|Braund, Mr. Owen ...|  male|22.0|    1|  NaN|       A/5 21171|   7.25|  N/A|       S|
|          2|    1|     1|Cumings, Mrs. Joh...|female|38.0|    1|  NaN|        PC 17599|71.2833|  C85|       C|
|          3|    1|     3|Heikkinen, Miss. ...|female|26.0|    0|  NaN|STON/O2. 3101282|  7.925|  N/A|       S|
|          4|    1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|  NaN|          113803|   53.1| C123|       S|
|          5|    0|     3|Allen, Mr. Willia...|  male|35.0|    0|  NaN|          373450|   8.05|  N/A|       S|
|          6|    0|     3|    Moran, Mr. James|  male|null|    0|  NaN|          330877| 8.4583|  N/A|  

dropna()
Función que elimina las filas con valores nulos de una columna en particular

In [ ]:
df = df.dropna(subset = "Embarked")
df.show()


#Sintaxis para eliminar todas las filas del dataset
#newdf = df.dropna()
#newdf.show()

+-----------+-----+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|label|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+-----+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|    0|     3|Braund, Mr. Owen ...|  male|22.0|    1|  NaN|       A/5 21171|   7.25|  N/A|       S|
|          2|    1|     1|Cumings, Mrs. Joh...|female|38.0|    1|  NaN|        PC 17599|71.2833|  C85|       C|
|          3|    1|     3|Heikkinen, Miss. ...|female|26.0|    0|  NaN|STON/O2. 3101282|  7.925|  N/A|       S|
|          4|    1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|  NaN|          113803|   53.1| C123|       S|
|          5|    0|     3|Allen, Mr. Willia...|  male|35.0|    0|  NaN|          373450|   8.05|  N/A|       S|
|          6|    0|     3|    Moran, Mr. James|  male|null|    0|  NaN|          330877| 8.4583|  N/A|  

Imputer 
Asignar a los valores null la media,mediana o la moda según cual sea mejor

In [ ]:
from pyspark.ml.feature import Imputer

imputer=Imputer(inputCols=["Age"],
                outputCols=["Age"])
model = imputer.fit(df)
df=model.transform(df)
df.show()

+-----------+-----+------+--------------------+------+-----------------+-----+-----+----------------+-------+-----+--------+
|PassengerId|label|Pclass|                Name|   Sex|              Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+-----+------+--------------------+------+-----------------+-----+-----+----------------+-------+-----+--------+
|          1|    0|     3|Braund, Mr. Owen ...|  male|             22.0|    1|  NaN|       A/5 21171|   7.25|  N/A|       S|
|          2|    1|     1|Cumings, Mrs. Joh...|female|             38.0|    1|  NaN|        PC 17599|71.2833|  C85|       C|
|          3|    1|     3|Heikkinen, Miss. ...|female|             26.0|    0|  NaN|STON/O2. 3101282|  7.925|  N/A|       S|
|          4|    1|     1|Futrelle, Mrs. Ja...|female|             35.0|    1|  NaN|          113803|   53.1| C123|       S|
|          5|    0|     3|Allen, Mr. Willia...|  male|             35.0|    0|  NaN|          373450|   8.05|  N/A|       S|


###6.- Crear columna calculadas para el dataframe 

In [ ]:
#FORMA1
from pyspark.sql.functions import expr
df = df.withColumn("Rango_edad",expr("case when Age < 20 then 'joven'"+
               "when Age < 30 then ' medio'"+
                   "when Age< 50 then 'adulto' else 'viejo' end"))
df.show()

+-----------+-----+------+--------------------+------+-----------------+-----+-----+----------------+-------+-----+--------+----------+
|PassengerId|label|Pclass|                Name|   Sex|              Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|Rango_edad|
+-----------+-----+------+--------------------+------+-----------------+-----+-----+----------------+-------+-----+--------+----------+
|          1|    0|     3|Braund, Mr. Owen ...|  male|             22.0|    1|  NaN|       A/5 21171|   7.25|  N/A|       S|     medio|
|          2|    1|     1|Cumings, Mrs. Joh...|female|             38.0|    1|  NaN|        PC 17599|71.2833|  C85|       C|    adulto|
|          3|    1|     3|Heikkinen, Miss. ...|female|             26.0|    0|  NaN|STON/O2. 3101282|  7.925|  N/A|       S|     medio|
|          4|    1|     1|Futrelle, Mrs. Ja...|female|             35.0|    1|  NaN|          113803|   53.1| C123|       S|    adulto|
|          5|    0|     3|Allen, Mr. Willia...| 

In [ ]:
df.show()

+-----------+-----+------+--------------------+------+-----------------+-----+-----+----------------+-------+-----+--------+----------+
|PassengerId|label|Pclass|                Name|   Sex|              Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|Rango_edad|
+-----------+-----+------+--------------------+------+-----------------+-----+-----+----------------+-------+-----+--------+----------+
|          1|    0|     3|Braund, Mr. Owen ...|  male|             22.0|    1|  NaN|       A/5 21171|   7.25|  N/A|       S|     medio|
|          2|    1|     1|Cumings, Mrs. Joh...|female|             38.0|    1|  NaN|        PC 17599|71.2833|  C85|       C|    adulto|
|          3|    1|     3|Heikkinen, Miss. ...|female|             26.0|    0|  NaN|STON/O2. 3101282|  7.925|  N/A|       S|     medio|
|          4|    1|     1|Futrelle, Mrs. Ja...|female|             35.0|    1|  NaN|          113803|   53.1| C123|       S|    adulto|
|          5|    0|     3|Allen, Mr. Willia...| 

In [ ]:
#FORMA2
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.functions import broadcast
from pyspark.sql.types import *
from pyspark.sql.functions import udf

df = df.selectExpr("*", ("case when Age < 20 then 'joven'"+
               "when Age < 30 then ' medio'"+
                   "when Age< 50 then 'adulto' else 'viejo' end as Rango_edad"))
df.show()


+-----------+-----+------+--------------------+------+-----------------+-----+-----------------+----------------+-------+-----+--------+----+--------+----------+
|PassengerId|label|Pclass|                Name|   Sex|              Age|SibSp|            Parch|          Ticket|   Fare|Cabin|Embarked|Sexo|Embarque|Rango_edad|
+-----------+-----+------+--------------------+------+-----------------+-----+-----------------+----------------+-------+-----+--------+----+--------+----------+
|          1|    0|     3|Braund, Mr. Owen ...|  male|             22.0|    1|1.596244131455399|       A/5 21171|   7.25|  N/A|       S| 0.0|     0.0|     medio|
|          2|    1|     1|Cumings, Mrs. Joh...|female|             38.0|    1|1.596244131455399|        PC 17599|712.833|  C85|       C| 1.0|     1.0|    adulto|
|          3|    1|     3|Heikkinen, Miss. ...|female|             26.0|    0|1.596244131455399|STON/O2. 3101282|  7.925|  N/A|       S| 1.0|     0.0|     medio|
|          4|    1|     1|Fu

In [ ]:
df.show()

+-----------+--------+------+--------------------+------+-----------------+-----+-----+----------------+-------+-----+--------+----------+
|PassengerId|Survived|Pclass|                Name|   Sex|              Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|Rango_edad|
+-----------+--------+------+--------------------+------+-----------------+-----+-----+----------------+-------+-----+--------+----------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|             22.0|    1|    0|       A/5 21171|   7.25|  N/A|       S|     medio|
|          2|       1|     1|Cumings, Mrs. Joh...|female|             38.0|    1|    0|        PC 17599|71.2833|  C85|       C|    adulto|
|          3|       1|     3|Heikkinen, Miss. ...|female|             26.0|    0|    0|STON/O2. 3101282|  7.925|  N/A|       S|     medio|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|             35.0|    1|    0|          113803|   53.1| C123|       S|    adulto|
|          5|       0|     

###7.- Convertir Variables categóricas a Númerica

In [ ]:
from pyspark.ml.feature import StringIndexer

df= StringIndexer(
    inputCol='Sex',
    outputCol='Sexo',
    handleInvalid='keep').fit(df).transform(df)

df= StringIndexer(
    inputCol='Embarked',
    outputCol='Embarque',
    handleInvalid='keep').fit(df).transform(df)


df= StringIndexer(
    inputCol='Rango_edad',
    outputCol='rangoA',
    handleInvalid='keep').fit(df).transform(df)
df.show()

+-----------+-----+------+--------------------+------+-----------------+-----+-----+----------------+-------+-----+--------+----------+----+--------+------+
|PassengerId|label|Pclass|                Name|   Sex|              Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|Rango_edad|Sexo|Embarque|rangoA|
+-----------+-----+------+--------------------+------+-----------------+-----+-----+----------------+-------+-----+--------+----------+----+--------+------+
|          1|    0|     3|Braund, Mr. Owen ...|  male|             22.0|    1|  NaN|       A/5 21171|   7.25|  N/A|       S|     medio| 0.0|     0.0|   0.0|
|          2|    1|     1|Cumings, Mrs. Joh...|female|             38.0|    1|  NaN|        PC 17599|71.2833|  C85|       C|    adulto| 1.0|     1.0|   1.0|
|          3|    1|     3|Heikkinen, Miss. ...|female|             26.0|    0|  NaN|STON/O2. 3101282|  7.925|  N/A|       S|     medio| 1.0|     0.0|   0.0|
|          4|    1|     1|Futrelle, Mrs. Ja...|female|    

Eliminar columnas categoricas

In [ ]:
df=df.drop('Sex')
df=df.drop('Rango_edad')
df=df.drop('Embarked')

In [ ]:

df.show()

+-----------+-----+------+--------------------+-----------------+-----+-----+----------------+-------+-----+----+--------+------+
|PassengerId|label|Pclass|                Name|              Age|SibSp|Parch|          Ticket|   Fare|Cabin|Sexo|Embarque|rangoA|
+-----------+-----+------+--------------------+-----------------+-----+-----+----------------+-------+-----+----+--------+------+
|          1|    0|     3|Braund, Mr. Owen ...|             22.0|    1|  NaN|       A/5 21171|   7.25|  N/A| 0.0|     0.0|   0.0|
|          2|    1|     1|Cumings, Mrs. Joh...|             38.0|    1|  NaN|        PC 17599|71.2833|  C85| 1.0|     1.0|   1.0|
|          3|    1|     3|Heikkinen, Miss. ...|             26.0|    0|  NaN|STON/O2. 3101282|  7.925|  N/A| 1.0|     0.0|   0.0|
|          4|    1|     1|Futrelle, Mrs. Ja...|             35.0|    1|  NaN|          113803|   53.1| C123| 1.0|     0.0|   1.0|
|          5|    0|     3|Allen, Mr. Willia...|             35.0|    0|  NaN|          373

###8.- Eliminar Variables que no inciden en el módelo

In [ ]:
df=df.drop('PassengerId')
df=df.drop('Name')
df=df.drop('Age')
df=df.drop('Ticket')
df=df.drop('Cabin')
df.show()

Py4JJavaError: ignored

Ver los tipos de datos para convertir columnas a númericos

In [ ]:
df.dtypes

[('label', 'int'),
 ('Pclass', 'int'),
 ('SibSp', 'int'),
 ('Parch', 'double'),
 ('Fare', 'double'),
 ('Sexo', 'double'),
 ('Embarque', 'double'),
 ('rangoA', 'double')]

Cambiar tipo de datos encaso de ser necesario

In [ ]:
from pyspark.sql.functions import col

df = df.select(col('Survived').cast('float'),                   
                         col('Pclass').cast('float'),
                         col('SibSp').cast('float'),
                         col('Parch').cast('float'),
                         col('Fare').cast('float'),
                         col('Sexo').cast('float'),
                         col('Embarque').cast('float'),
                         col('rangoA').cast('float')
                        )
df.dtypes 

###9.- Creación Vector Assembler
vector assembler
agrupamiento de datos
Escalado y normalización
trabajan con caracteristicas categóricas


In [ ]:
from pyspark.ml.feature import VectorAssembler
caracteristicas=['Pclass',
                 'SibSp',
                 'Parch',
                 'Fare',
                 'Sexo',
                 'Embarque',
                 'rangoA'
                 ]
assembler= VectorAssembler(inputCols=caracteristicas,outputCol='features')
df=assembler.transform(df)

In [ ]:
df.show()

###10.- Normalización de las variables
Permite una distribucuión normal

In [ ]:
from pyspark.ml.feature import Normalizer
from pyspark.ml.linalg import Vectors
normalizer = Normalizer(inputCol="features", outputCol="Features_N", p=1.0)
df = normalizer.transform(df)
df.show()

+-----+------+-----+-----+-------+----+--------+------+--------------------+--------------------+
|label|Pclass|SibSp|Parch|   Fare|Sexo|Embarque|rangoA|            features|          Features_N|
+-----+------+-----+-----+-------+----+--------+------+--------------------+--------------------+
|  0.0|   3.0|  1.0|  0.0|   7.25| 0.0|     0.0|   0.0|(7,[0,1,3],[3.0,1...|(7,[0,1,3],[0.266...|
|  1.0|   1.0|  1.0|  0.0|71.2833| 1.0|     1.0|   1.0|[1.0,1.0,0.0,71.2...|[0.01310902870950...|
|  1.0|   3.0|  0.0|  0.0|  7.925| 1.0|     0.0|   0.0|(7,[0,3,4],[3.0,7...|(7,[0,3,4],[0.251...|
|  1.0|   1.0|  1.0|  0.0|   53.1| 1.0|     0.0|   1.0|[1.0,1.0,0.0,53.0...|[0.01751313531914...|
|  0.0|   3.0|  0.0|  0.0|   8.05| 0.0|     0.0|   1.0|(7,[0,3,6],[3.0,8...|(7,[0,3,6],[0.248...|
|  0.0|   3.0|  0.0|  0.0| 8.4583| 0.0|     2.0|   0.0|(7,[0,3,5],[3.0,8...|(7,[0,3,5],[0.222...|
|  0.0|   1.0|  0.0|  0.0|51.8625| 0.0|     0.0|   3.0|(7,[0,3,6],[1.0,5...|(7,[0,3,6],[0.017...|
|  0.0|   3.0|  3.0|

In [ ]:
df=df.drop('features')
df.show()

+-----+------+-----+-----+-------+----+--------+------+--------------------+
|label|Pclass|SibSp|Parch|   Fare|Sexo|Embarque|rangoA|          Features_N|
+-----+------+-----+-----+-------+----+--------+------+--------------------+
|  0.0|   3.0|  1.0|  0.0|   7.25| 0.0|     0.0|   0.0|(7,[0,1,3],[0.266...|
|  1.0|   1.0|  1.0|  0.0|71.2833| 1.0|     1.0|   1.0|[0.01310902870950...|
|  1.0|   3.0|  0.0|  0.0|  7.925| 1.0|     0.0|   0.0|(7,[0,3,4],[0.251...|
|  1.0|   1.0|  1.0|  0.0|   53.1| 1.0|     0.0|   1.0|[0.01751313531914...|
|  0.0|   3.0|  0.0|  0.0|   8.05| 0.0|     0.0|   1.0|(7,[0,3,6],[0.248...|
|  0.0|   3.0|  0.0|  0.0| 8.4583| 0.0|     2.0|   0.0|(7,[0,3,5],[0.222...|
|  0.0|   1.0|  0.0|  0.0|51.8625| 0.0|     0.0|   3.0|(7,[0,3,6],[0.017...|
|  0.0|   3.0|  3.0|  1.0| 21.075| 0.0|     0.0|   2.0|[0.09975062091093...|
|  1.0|   3.0|  0.0|  2.0|11.1333| 1.0|     0.0|   0.0|[0.17509761868356...|
|  1.0|   2.0|  1.0|  0.0|30.0708| 1.0|     1.0|   2.0|[0.05395081729692...|

##5.- Crear datos de entramiento y test 

In [ ]:
(train,test) = df.randomSplit([0.8,0.2])

#6 Creción, entrenamiento y Evaluación de Modelos



## 1.-Modelo Random Forest
Random Forest o Bosques Aleatorios,consiste en crear muchos árboles para luego utilizarlos en la predicción de la variable de interés.

###1.- Creación Modelo 

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(labelCol='label', 
                            featuresCol='Features_N',
                            maxDepth=5)

In [ ]:
modelo = rf.fit(train)

In [ ]:
# Predict with the test dataset
predictions = modelo.transform(test)

### 2.- Evaluación Modelo RandomForest

Métrica Acuuracy o Exactitud

In [ ]:
# Evaluate our model
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator2 = MulticlassClassificationEvaluator(labelCol='label', metricName='accuracy')
acura =evaluator2.evaluate(predictions)
print('Test acuracy = ', acura)

Test acuracy =  0.8058823529411765


Métrica F1

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator1 = MulticlassClassificationEvaluator(labelCol='label', metricName='f1')
f1 = evaluator1.evaluate(predictions)
print('Test f1 = ', f1)

Test f1 =  0.8025903939557475


## 2.-Modelo Árbol de Decisión

###1.- Creación Modelo 
Los árboles de decisión tienen como objetivo predecir la variable respuesta  Y en función de covariables.

In [ ]:
from pyspark.ml.classification import  DecisionTreeClassifier
Tree = DecisionTreeClassifier (labelCol='label', 
                            featuresCol='Features_N')

In [ ]:
modelo = Tree.fit(train)

In [ ]:
predictions = modelo.transform(test)

###2.- Evaluación Modelo Árbol de decisión


Métrica Acuuracy o Exactitud

In [ ]:
# Evaluate our model
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator2 = MulticlassClassificationEvaluator(labelCol='label', metricName='accuracy')
acura =evaluator2.evaluate(predictions)
print('Test acuracy = ', acura)

Test acuracy =  0.8323699421965318


Métrica F1

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator1 = MulticlassClassificationEvaluator(labelCol='label', metricName='f1')
f1 = evaluator1.evaluate(predictions)
print('Test f1 = ', f1)

Test f1 =  0.8286322844217942


##3.- Modelo LinearSVC


1.- Creación Modelo
El método Clasificador de vectores de soporte lineal 

In [ ]:
from pyspark.ml.classification import LinearSVC
svc = LinearSVC(featuresCol='Features_N', labelCol='label')
modelo = svc.fit(train)

In [ ]:
predictions = modelo.transform(test)

###2.- Evaluación Modelo 

Métrica Acuuracy o Exactitud

In [ ]:
# Evaluate our model
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator2 = MulticlassClassificationEvaluator(labelCol='label', metricName='accuracy')
acura =evaluator2.evaluate(predictions)
print('Test acuracy = ', acura)

Test acuracy =  0.7630057803468208


Métrica F1

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator1 = MulticlassClassificationEvaluator(labelCol='label', metricName='f1')
f1 = evaluator1.evaluate(predictions)
print('Test f1 = ', f1)

Test f1 =  0.7683804781139247


##4.- Modelo NaiveBayes

###1.- Creación Modelo

In [ ]:
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes(featuresCol='Features_N', labelCol='label')
modelo = nb.fit(train)



In [ ]:
predictions = modelo.transform(test)

###2.- Evaluación Modelo

Métrica Acuuracy o Exactitud

In [ ]:
# Evaluate our model
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator2 = MulticlassClassificationEvaluator(labelCol='label', metricName='accuracy')
acura =evaluator2.evaluate(predictions)
print('Test acuracy = ', acura)

Test acuracy =  0.6589595375722543


Métrica F1

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator1 = MulticlassClassificationEvaluator(labelCol='label', metricName='f1')
f1 = evaluator1.evaluate(predictions)
print('Test f1 = ', f1)

Test f1 =  0.5234939880364947


##5.- Modelo Regreasión Lógistica

###1.-Creación Modelo

In [ ]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol='Features_N', labelCol='label')
modelo = lr.fit(train)
predictions = modelo.transform(test)

###2.- Evaluación Módelo

Métrica Acuuracy o Exactitud

In [ ]:
# Evaluate our model
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator2 = MulticlassClassificationEvaluator(labelCol='label', metricName='accuracy')
acura =evaluator2.evaluate(predictions)
print('Test acuracy = ', acura)

Test acuracy =  0.7630057803468208


Métrica F1

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator1 = MulticlassClassificationEvaluator(labelCol='label', metricName='f1')
f1 = evaluator1.evaluate(predictions)
print('Test f1 = ', f1)

Test f1 =  0.7676000939960534


##6.- Modelo BTClassifier



###1.- Creación Módelo

In [ ]:
from pyspark.ml.classification import GBTClassifier
gbt = GBTClassifier(featuresCol='Features_N', labelCol='label')
modelo = lr.fit(train)
predictions = modelo.transform(test)

###2:_ Evaluación Modelo

In [ ]:
# Evaluate our model
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator2 = MulticlassClassificationEvaluator(labelCol='label', metricName='accuracy')
acura =evaluator2.evaluate(predictions)
print('Test acuracy = ', acura)

Test acuracy =  0.7630057803468208


In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator1 = MulticlassClassificationEvaluator(labelCol='label', metricName='f1')
f1 = evaluator1.evaluate(predictions)
print('Test f1 = ', f1)

Test f1 =  0.7676000939960534


##7.- Modelo MultilayerPerceptron

###1.- Creación Modelo

In [ ]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
mpc = MultilayerPerceptronClassifier(featuresCol='Features_N', labelCol='label')
mpc_model = gbt.fit(train)

###2.- Evaluación Modelo

In [ ]:
# Evaluate our model
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator2 = MulticlassClassificationEvaluator(labelCol='label', metricName='accuracy')
acura =evaluator2.evaluate(predictions)
print('Test acuracy = ', acura)

Test acuracy =  0.7630057803468208


In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator1 = MulticlassClassificationEvaluator(labelCol='label', metricName='f1')
f1 = evaluator1.evaluate(predictions)
print('Test f1 = ', f1)

Test f1 =  0.7676000939960534
